# RNN to generate text from GOT

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [2]:

text = open('gameofthrones.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# Take a look at the first 250 characters in text
print(text[:250])

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 5662324 characters


“We should start back,” Gared urged as the woods began to grow dark around them. “The wildlings are dead.”

“Do the dead frighten you?” Ser Waymar Royce asked with just the hint of a smile.

Gared did not rise to the bait. He was an old man, past f
86 unique characters


In [3]:
# Creating a mapping from unique characters to indices
char2idx = {char : index for index, char in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

{
  'f' :  55,
  '—' :  80,
  'z' :  75,
  '“' :  83,
  '/' :   8,
  'Z' :  47,
  ' ' :   1,
  'H' :  29,
  '1' :  10,
  'K' :  32,
  'b' :  51,
  'q' :  66,
  '’' :  82,
  'X' :  45,
  'W' :  44,
  'E' :  26,
  'm' :  62,
  'x' :  73,
  '8' :  17,
  '!' :   2,
  ...
}
'\n\n“We should ' ---- characters mapped to int ---- > [ 0  0 83 44 54  1 68 57 64 70 61 53  1]


In [4]:
# The maximum length sentence we want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# Creates a Dataset whose elements are slices of the given tensors.

# Splits the dataset in batches of size seq_length
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

Instructions for updating:
Colocations handled automatically by placer.
'\n\n“We should start back,” Gared urged as the woods began to grow dark around them. “The wildlings are dead.”\n\n“Do the dead frighten you?” Ser Waymar Royce asked with just the hint of a smile.\n\nGared di'
'd not rise to the bait. He was an old man, past fifty, and he had seen the lordlings come and go. “Dead is dead,” he said. “We have no business with the dead.”\n\n“Are they dead?” Royce asked softly. “Wh'
'at proof have we?”\n\n“Will saw them,” Gared said. “If he says they are dead, that’s proof enough for me.”\n\nWill had known they would drag him into the quarrel sooner or later. He wished it had been late'
'r rather than sooner. “My mother told me that dead men sing no songs,” he put in.\n\n“My wet nurse said the same thing, Will,” Royce replied. “Never believe anything you hear at a woman’s tit. There are '
'things to be learned even from the dead.” His voice echoed, too loud in the twilit forest.\n\n“We

In [5]:
# Generate the input and target texts
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print ('Input data: ' + repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:' + repr(''.join(idx2char[target_example.numpy()])))

Input data: '\n\n“We should start back,” Gared urged as the woods began to grow dark around them. “The wildlings are dead.”\n\n“Do the dead frighten you?” Ser Waymar Royce asked with just the hint of a smile.\n\nGared d'
Target data:'\n“We should start back,” Gared urged as the woods began to grow dark around them. “The wildlings are dead.”\n\n“Do the dead frighten you?” Ser Waymar Royce asked with just the hint of a smile.\n\nGared di'


In [6]:
# Goes through the 5 first steps of the first sequence
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 83 ('“')
Step    2
  input: 83 ('“')
  expected output: 44 ('W')
Step    3
  input: 44 ('W')
  expected output: 54 ('e')
Step    4
  input: 54 ('e')
  expected output: 1 (' ')


In [7]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

In [8]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [9]:
import functools
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True), 
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=len(vocab), 
    embedding_dim=embedding_dim, 
    rnn_units=rnn_units, 
    batch_size=BATCH_SIZE)

In [10]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()

(64, 200, 86) # (batch_size, sequence_length, vocab_size)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           22016     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 86)            88150     
Total params: 4,045,398
Trainable params: 4,045,398
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Decode the prediction of the model
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print('Input: \n' + repr(''.join(idx2char[input_example_batch[0]])))
print('')
print('Next Char Predictions: \n' + repr("".join(idx2char[sampled_indices ])))

Instructions for updating:
Use tf.random.categorical instead.
Input: 
'sailed ages ago.\n\nTwo of the guardsmen were dicing together while the third walked rounds, his hand on the pommel of his sword. Ashamed to let them see her crying like a baby, she stopped to rub at he'

Next Char Predictions: 
'KD)6xn:’edêklaruJNP-Hgh30lA5s[iPU5mCM]u9fjwebWGYHV(6kZMzOlLW‘r-Z:KS8F‘}6h}5f.3uo)zs}(0IVtv)SpE—[]ZmV”lOyHu8fk6THDOx6,!ps‘C\nQ:zLJ 9ytK-I5f0((?Jdf3…WucY“tL/—c69lN/cYn:cF]wJ.{Y5kr-m)ns(Pva F5LEi9R4?k —Bn'


In [12]:
def loss(labels, logits):
  return tf.keras.backend.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: " + str(example_batch_predictions.shape) + " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      " + str(example_batch_loss.numpy().mean()))

Prediction shape: (64, 200, 86) # (batch_size, sequence_length, vocab_size)
scalar_loss:      4.4535155


In [13]:
model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [14]:
EPOCHS=15

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/15
441/442 [============================>.] - ETA: 0s - loss: 2.1226WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
442/442 [==============================] - 106s 240ms/step - loss: 2.1213
Epoch 2/15
442/442 [==============================] - 103s 234ms/step - loss: 1.4204
Epoch 3/15
442/442 [==============================] - 104s 234ms/step - loss: 1.2714
Epoch 4/15
442/442 [==============================] - 103s 234ms/step - loss: 1.2057
Epoch 5/15
442/442 [==============================] - 103s 232ms/step - loss: 1.1651
Epoch 6/15
442/442 [==============================] - 103s 233ms/step - loss: 1.1347
Epoch 7/15
442/442 [============================

In [15]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 86)             88150     
Total params: 4,045,398
Trainable params: 4,045,398
Non-trainable params: 0
_________________________________________________________________


In [16]:
def generate_text(model, start_string, num_generate=1000):
  # Evaluation step (generating text using the learned model)
  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [17]:
print(generate_text(model, start_string=u'Jon'))

Jon reminded herself trying to open the surface of the feast meals. They are worse than you can believe that the steward serve me, she told herself.

Three hundred he had liked. She says with this falseheld smile as I do, Bran. Brother. The master of cities all night, Jon?”

“Not that I suffered that or will she broken you, nepreed. Lord Alester said he needed to hear after it is, blue as bruise.”

Sansa seurting the dripped face, he plurped the brill as he loosed. Snow might have killed anything I had to kill him. He had no time for a fawn burning beside this damned black-girl ring, trade three-hundred red eyes, and use was someone else more auling another she stones that night.

He had been watching him, and to men he would have the peffice of Lord Eddard, unnisted to spy at her wrist. “Your crimes,” she told them. “Bastard of sixter, you did nothank your father’s heir, an axe ill.”

Ned had taken how it began to smile. “Have you ever wanted anyn called. They send no matter what they

In [18]:
print(generate_text(model, start_string=u'Robb'))

Robberch of the Seven Kingdoms.” The knight stroked her cheek cletched and galloping loose sourleaf of the riderlass. It was hard to take his meat hissing, not wet and awkward, but Littlefinger had been ribbong up, and even the biggest she was… Cersei will like to kill her?”

Dany could not have called him as he chase of him.

The black cloak was shr fierce that Sam had a moment, lean and among their laps. “I shall look in his grief, his fighting wailed to the core from the mouth to following.

“Four here will your chosen you won’t be safe for Ned Stark, very kind. What come beyond crows, from the roof of the realm. We’re left.”

“I didn’t serve huntry in King’s Landing,” the boy said. “Whe wants the desert out where I am done, to present you that he is my brother, Ser were you. Seven are coming on!”

Khal Drogo grumbled at the morning archers, as if tringing Tyrion had horses.

They jumped up out of the castle wall, clad in anxious was half-catched by camp followers, and when they wer

In [19]:
print(generate_text(model, start_string=u'Eddard'))

Eddard said, “Ser Ryman, then men are broken, not trying not, and ride back to Asshai, ser.”

“In?” he said. “That was said, and the Hands come to King Robert’s Rebellion, for the man’s scars.

At first, the pursue was likely till now? I thought they do not like they, the prayers. Ser Red cleared rain and waggled plate. With Ser Whalen Frey Tyrell s of darkness beneath the reddalt, teeth stretched before long sharp sun silences. After the size Lannister gathered before the vast finger was pale orange icorrides, if they were not every little man in a stick.

“Nothing to fight these pleasures. And yet… cut off. Even their corpses might teach me what you need to call orders. Renly rather peetle flinking into Lord White Harbor, and Unsullied in the Seven Kingdoms, she thought she had killed her sister. She turned to Dater a killer. “I shall die in my eyes…” Bores were left. “Will I miss mine.”

“Your return…” Jon admitted, staring back. “I have wrapped your tent back to the morrow, or watc

In [20]:
print(generate_text(model, start_string=u'Cersei'))

Cersei understood that Khal Drogo’s flesh… they are convenient ago. His lordship was welcome to her, but the mea-they’s breath saw Kraznys thick sullen stone of the mast, golden embroidered with soft, alongside casules, walls and armor, dangerous, the window, we are holding maidens and listen to overhousands. “Thould take some merchant innocent right take now?”

Sansa loaned his head against a few mounted with red. They gave her at them and said, “We are not telling her?”

“Smoke to the night, him and children but the snow. Neither men have the less, and eaten.”

“She is my master’ every day after me,” he said.

Arya was pulling after her; Varys was failing, his own hand clenched.

“Where she told Davos we’ve hair and seed, whether she was heard,” declared Lord Tywin. “So you will wish I could…”

“She looked up with Prince Tommen, when she saw the gate shadow from its exterior. They someteard is not there. Ser Jorah Mormont had smashed past her men amused herself. “This is a father oul

In [21]:
print(generate_text(model, start_string=u'wildling'))

wildling, his cheeks were cragged at the body. “Shadows is gone and the father told me, when it is the sea, his castle will be wide behind his breasts. “The shadows come to Riverrun,” the Lord Commander offered softly. “Maybe Robb, the venisan Drogo is rich and rich in the gallery. Ser Loras is innocent, Father? Perhaps I should do with you. You request spells to frog a maester’s cony. Evin there is nothing.”

Tyrion shook his head. “Or my man-ons,” the man made a face of Rattleshirt room as his mouth trying to feel Ser Meryn Trant sensed than ten, fortyffinger came down on him, but there wine and the heavy lank where she heard it music layer, past ith thick were cracked and stroked her eyes to his breath when Arya licked her away. “Jory wad Robb and after Lysa Arryn.”

“The queen fears your new?”

“I am his father’s aple, Kingslayer.”

“They hear me lide,” he said.

“I’m afraid I desperate wrought and let this owen than Varys. All about him, he had to look so little, and why, not if h

In [22]:
print(generate_text(model, start_string=u'Kingslayer'))

Kingslayer, and in his mother, LIrd Walder Freys are let his shoes seem to warm her accustoms,” Rhoen said. “You won’t have the Lady Brienne.”

“If you were lacion or snarks, men will enders will rule the bravos to return in shattered swords.

Dany paused to swing it. The pounding, the only thing that Blowfore suitseated the Blackfish. He did not have the council the or they’d provek a few withered armor.”

“Sad lie. Toge I’ll make a hundred scothe is,” Ser Rodrik commented sullenly to break the cup at Duskendale.

“I didn’t keep him now.”.

Two beasts were pointing through arrows along the blankets, drawing some blood to long aforonous while I needed every visits of yours.” Ten, how it hadn’t been sold from that god no party and hear, not carefully. Only then did they have soon enough to struggle when they made good nightful. Gods did not really know Jon. Not crushing Dolorous Edd Satin. As his and her name can sit, he knew it so not there must know. Your ransom walked off the gates. 